# 학습 관련 기술들

## 매개변수 갱신

### 모험가 이야기
### 확률적 경사 하강법(SGD)

In [1]:
class SGD:
    def __init__(self, lr = 0.01):
        self.lr = lr
        
    def update(self, params, grads):
        for key in params.keys():
            params[key] -= self.lr * grads[key]

* SGD 클래스를 활용한 신경망 매개변수 진행

        network = TwoLayerNet(...)
        optimizer = SGD()

        for i in range(10000):
            ...
            x_batch, t_batch = get_mini_batch(...) # 미니 배치
            grads = network.gradient(x_batch, t_batch)
            params = network.params
            optimizer.update(params, grads)
            ...
* optimizer : 최적화를 행하는 자
SGD()가 그 역할을 한다.  
매개변수 갱신을 optimizer가 하고, 사용자가 매개변수와 기울기 정보를 optimizer에 입력한다.
* 최적화 담당 클래스를 분리해서 구현하면 모듈화 하기 좋다.  
optimizer = SGD()  
optimizer = Momentum() : SGD가 모멘텀으로 바뀐다.
* Lasagne 딥러닝 프레임워크 : updates.py 에 최적화 기법을 구현.

### SGD의 단점
문제에 따라 비효율적일 때가 있다.  
다음 함수의 최솟값 구하는 문제
$$f(x,y) = {1 \over 20}x^2 + y^2$$

* 함수의 그래프(왼쪽), 등고선(오른쪽)
![img](./deep_learning_images/fig_6-1.png)

* 함수의 기울기
![img](./deep_learning_images/fig_6-2.png)
y축 방향이 크고, x축 방향은 작다.  

* 주의할 점
    * 최솟값이 되는 장소는 $(x,y) = (0,0)$이지만 그림의 기울기는 대부분 원점을 가리키지 않는다.
    
* SGD 적용  
초깃값을 $(x,y) = (-7.0,2.0)$으로 지정한다.
![img](./deep_learning_images/fig_6-3.png)
최솟값인 원점까지 지그재그로 이동한다. 비효율적이다.  
심하게 굽어진 움직임을 보여준다.  
* SGD 단점
    * 비등방성(anisotropy) 함수(기울기가 달라지는 함수)에서 탐색 경로가 비효율적이다.
    * 근본 원인 : 기울어진 방향이 본래의 최솟값과 다른 방향을 가리킨다.
    
* 개선한 방법
    * 모멘텀, AdaGrad, Adam 소개

### 모멘텀
모멘텀(momentum)

In [4]:
class Momentum:
    def __init__(self, lr = 0.01, momentum = 0.9):
        self.lr = lr
        self.momentum = momentum
        self.v = None #v : 물체의 속도
        
    def update(self, params, grads):
        if self.v is None:
            self.v = {}
            for key, val in params.items():
                self.v[key] = np.zeros_like(val)
                
            for key in params.keys():
                self.v[key] = self.momentum*self.v[key] - self.lr*grads[key]
                params[key] += self.v[key]

모멘텀에 의한 최적화 갱신 경로
![img](./deep_learning_images/fig_6-5.png)
SGD 보다 지그재그 정도가 덜하다.  
> x축의 힘이 아주 작지만 방향이 변하지 않아서, 한 방향으로 일정하게 가속하기 때문이다.

### AdaGrad

$$\mathbf{h} \leftarrow \mathbf{h} + {\partial L \over \partial \mathbf{W}} \odot {\partial L \over \partial \mathbf{W}} \\
\mathbf{W} \leftarrow \mathbf{W} - \eta{1 \over \sqrt{\mathbf{h}}} {\partial L \over \partial \mathbf{W}}$$

In [6]:
class AdaGrad:
    def __init__(self, lr = 0.01):
        self.lr = lr
        self.h = None
        
    def update(self, params, grads):
        if self.h is None:
            self.h = {}
            for key, val in params, items():
                self.h[key] = np.zeros_like(val)
                
        for key in params.keys():
            self.h[key] += grads[key] * grads[key]
            params[key] -= self.lr * grads[key] / (np.sqrt(self.h[key]) + 1e-7)

![img](./deep_learning_images/fig_6-6.png)



### Adam
모멘텀과 AdaGrad 기법을 융합한 것이 Adam이다.
![img](./deep_learning_images/fig_6-7.png)


### 어느 갱신 방법을 사용할 것인가?

In [14]:
!python ./optimizer_compare_naive.py

Figure(640x480)
